In [17]:
#import necessary packages
import pandas as pd
import numpy as np
import os
from urllib.request import urlopen
import json
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.interpolate import griddata

Using the GBIF API to download iNaturalist data

Jillian: Time range / time series of species

Erica: spatial distribution of a species 
Kiran: Synthesizing iNaturalist data (counts of observations?)


In [ ]:
import pygbif
#need to run pip install pygbif in terminal

In [ ]:
documentaton

doc

Using iNaturalist Research - Grade Observations
https://www.gbif.org/dataset/50c9509d-22c7-4a22-a47d-8c48425ef4a7

Help for using the API: https://www.gbif.org/developer/summary



In [ ]:
# Read in your token
#iucn_token="9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee"

# Root URL for GBIF API
gbif_url=" https://api.gbif.org/v1/"

#don't need a token, just your username and password for your GBIF account

In [1]:
from pygbif import species
species.name_suggest(q='Pygoscelis adeliae')

#or look up within the species module using any of these commands:
#species.name_backbone(rank=None, kingdom=None, phylum=None, clazz=None, order=None, family=None, genus=None, strict=False, verbose=False, offset=None, limit=100, **kwargs)


# usrname = "erica-bishop-ucsb"
# password = "p@$$word"

[{'key': 2481663,
  'nameKey': 9561926,
  'kingdom': 'Animalia',
  'phylum': 'Chordata',
  'order': 'Sphenisciformes',
  'family': 'Spheniscidae',
  'genus': 'Pygoscelis',
  'species': 'Pygoscelis adeliae',
  'kingdomKey': 1,
  'phylumKey': 44,
  'classKey': 212,
  'orderKey': 7190978,
  'familyKey': 5284,
  'genusKey': 2481662,
  'speciesKey': 2481663,
  'parent': 'Pygoscelis',
  'parentKey': 2481662,
  'nubKey': 2481663,
  'scientificName': 'Pygoscelis adeliae (Hombron & Jacquinot, 1841)',
  'canonicalName': 'Pygoscelis adeliae',
  'rank': 'SPECIES',
  'status': 'ACCEPTED',
  'synonym': False,
  'higherClassificationMap': {'1': 'Animalia',
   '44': 'Chordata',
   '212': 'Aves',
   '7190978': 'Sphenisciformes',
   '5284': 'Spheniscidae',
   '2481662': 'Pygoscelis'},
  'class': 'Aves'},
 {'key': 10492546,
  'nameKey': 9561933,
  'kingdom': 'Viruses',
  'phylum': 'Cossaviricota',
  'order': 'Sepolyvirales',
  'family': 'Polyomaviridae',
  'genus': 'Gammapolyomavirus',
  'species': 'Pygo

In [ ]:
#Lets explore this API investigating penguins!
# Look up names like mammalia
species.name_lookup(q='Spheniscidae')

# Paging
species.name_lookup(q='Spheniscidae', limit=1)
species.name_lookup(q='Spheniscidae', limit=1, offset=2)

# large requests, use offset parameter
first = species.name_lookup(q='Spheniscidae', limit=1000)
second = species.name_lookup(q='Spheniscidae', limit=1000, offset=1000)

# Get all data and parse it, removing descriptions which can be quite long
species.name_lookup('Pygoscelis adeliae', rank="species", verbose=True)

# Get all data and parse it, removing descriptions field which can be quite long
out = species.name_lookup('Pygoscelis adeliae', rank="species")
res = out['results']
[ z.pop('descriptions', None) for z in res ]
res

# Fuzzy searching
# species.name_lookup(q='Heli', rank="genus")

# # Limit records to certain number
# species.name_lookup('Helianthus annuus', rank="species", limit=2)

# # Query by habitat
# species.name_lookup(habitat = "terrestrial", limit=2)
# species.name_lookup(habitat = "marine", limit=2)
# species.name_lookup(habitat = "freshwater", limit=2)

# Using faceting
# species.name_lookup(facet='status', limit=0, facetMincount='70000')
# species.name_lookup(facet=['status', 'higherTaxonKey'], limit=0, facetMincount='700000')

# species.name_lookup(facet='nameType', limit=0)
# species.name_lookup(facet='habitat', limit=0)
# species.name_lookup(facet='datasetKey', limit=0)
# species.name_lookup(facet='rank', limit=0)
# species.name_lookup(facet='isExtinct', limit=0)

# text highlighting
species.name_lookup(q='plant', hl=True, limit=30)

# Lookup by datasetKey
species.name_lookup(datasetKey='3f8a1297-3259-4700-91fc-acc4170b27ce')

In [4]:
test_data_1 = species.name_lookup(institution_code="iNaturalist", q='Pygoscelis adeliae')
test_data_2 = species.name_lookup(q='Pygoscelis adeliae')
test_data_1 == test_data_2

False

In [8]:
from pygbif import occurrences as occ

In [10]:
adelie_iNat = occ.search(institutionCode="iNaturalist", limit=20)

In [13]:
type(adelie_iNat)

dict

In [18]:
adelie_data = occ.search(institutionCode="iNaturalist", 

ValueError: All arrays must be of the same length

In [ ]:
help(from

In [19]:


adelie_data = dict(zip(splist, 

{'offset': 0,
 'limit': 20,
 'endOfRecords': False,
 'count': 53352487,
 'results': [{'key': 3455477406,
   'datasetKey': '50c9509d-22c7-4a22-a47d-8c48425ef4a7',
   'publishingOrgKey': '28eb1a3f-1c15-4a95-931a-4af90ecb574d',
   'installationKey': '997448a8-f762-11e1-a439-00145eb45e9a',
   'publishingCountry': 'NZ',
   'protocol': 'DWC_ARCHIVE',
   'lastCrawled': '2022-11-17T12:13:40.744+00:00',
   'lastParsed': '2022-11-17T17:01:00.472+00:00',
   'crawlId': 325,
   'hostingOrganizationKey': '28eb1a3f-1c15-4a95-931a-4af90ecb574d',
   'extensions': {'http://rs.gbif.org/terms/1.0/Multimedia': [{'http://purl.org/dc/terms/license': 'http://creativecommons.org/licenses/by/4.0/',
      'http://purl.org/dc/terms/references': 'https://www.inaturalist.org/photos/174328292',
      'http://purl.org/dc/terms/rightsHolder': 'nick_goldwater',
      'http://rs.tdwg.org/dwc/terms/catalogNumber': '174328292',
      'http://purl.org/dc/terms/created': '2021-12-31T23:46:01Z',
      'http://purl.org/dc/ter